## Test Run 2: Extract Meaningful Span

This notebook continues from Test Run 1, but defines *Span* more meaningfully. We want to achieve that, 1 *Sentence* = 1+ *Spans* = 1+ *Clauses*. After the default `Ngram()` class, we customize Matcher to exclude any span with comma.

Note that in this test run, we will initiate `SnorkelSession()` without re-create `Document` or `Sentence`, which has been created in test run 1.


In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os

from snorkel import SnorkelSession
from snorkel.parser.spacy_parser import Spacy
from snorkel.parser import CorpusParser
from snorkel.models import Document, Sentence

session = SnorkelSession()
print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

sents = session.query(Sentence).all()
n_max_corpus=0
for sent in sents:
    n_max_corpus=max(n_max_corpus,len(sent.words))

print("The longest sentence has "+str(n_max_corpus)+" tokens.")

from snorkel.models import Document
from util import number_of_people

docs = session.query(Document).order_by(Document.name).all()

train_sents = set()
dev_sents   = set()
test_sents  = set()

for i, doc in enumerate(docs):
    for s in doc.sentences:
        if i % 10 == 8:
            dev_sents.add(s)
        elif i % 10 == 9:
            test_sents.add(s)
        else:
            train_sents.add(s)


Documents: 1000
Sentences: 4487
The longest sentence has 101 tokens.


In test run 1, we have tried using **single** `DictionaryMatcher`. Now it's time to explore more, e.g. combining **multiple** `DictionaryMatchers`. 

Fortunately, we could combine them through `Union(Matcher)` (takes the union of candidate sets returned by child matcher), `Intersection(Matcher)`, or `Concat(Matcher)` (candidates which are the concatenation of adjacent matches from child matchers).

In [4]:
from snorkel.models import candidate_subclass
from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import *

Background = candidate_subclass('Background', ['background_cue'])

ngrams = Ngrams(n_max=n_max_corpus) # we define the maximum n value as n_max_corpus

non_comma_matcher=DictionaryMatch(d=[','],longest_match_only=True,reverse=True)  
background_matcher=DictionaryMatch(d=['previous','motivated','recent','widely'],longest_match_only=True) 

non_comma_background_matcher=CandidateExtractor(Background, [ngrams], [Intersection(non_comma_matcher,background_matcher)])

In [5]:
for i, sents in enumerate([train_sents, dev_sents, test_sents]):
    %time non_comma_background_matcher.apply(sents, split=i)
    print("Split "+str(i)+" - number of candidates extracted:", session.query(Background).filter(Background.split == i).count(),"\n\n")
            

Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 8.79 s, sys: 284 ms, total: 9.08 s
Wall time: 9.18 s
Split 0 - number of candidates extracted: 106 


Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 1.42 s, sys: 144 ms, total: 1.57 s
Wall time: 1.57 s
Split 1 - number of candidates extracted: 11 


Clearing existing...
Running UDF...
[========================================] 100%

CPU times: user 1.47 s, sys: 148 ms, total: 1.62 s
Wall time: 1.63 s
Split 2 - number of candidates extracted: 9 




In [7]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

cands = session.query(Background).filter(Background.split == 0).all()

document_list=list()
for i in range(len(cands)):
    printmd("**"+str(i)+"/"+str(len(cands))+" Candidate/Span:**\t`"+str(cands[i].background_cue)+"`")
    printmd("**Its parent Sentence's text:**\t"+str(cands[i].get_parent().text))
    print()
    

**0/106 Candidate/Span:**	`Span("b'we investigate how to use machine learning to add reactivity to a previously \n'", sentence=2476, chars=[15,92], words=[4,17])`

**Its parent Sentence's text:**	In this paper, we investigate how to use machine learning to add reactivity to a previously 


**1/106 Candidate/Span:**	`Span("b'issues of material embodiment and environmental embedding have emerged as important areas of research attention over recent decades.'", sentence=4295, chars=[33,164], words=[7,25])`

**Its parent Sentence's text:**	Within the sciences of the mind, issues of material embodiment and environmental embedding have emerged as important areas of research attention over recent decades.

**2/106 Candidate/Span:**	`Span("b'and three approaches are presented for distinguishing amongst objects from a previously-encountered set: The first approach is entirely geometric in nature and borrows ideas from "\n'", sentence=4197, chars=[32,212], words=[6,35])`

...

**Its parent Sentence's text:**	In this lecture we combine ideas from the previous two lectures, linear monadic logic programming and higher-order abstract syntax, to present a specification technique for programming languages we call substructural operational semantics.

**104/106 Candidate/Span:**	`Span("b'As sketched in the previous lecture'", sentence=1077, chars=[0,34], words=[0,5])`

**Its parent Sentence's text:**	As sketched in the previous lecture, this an important piece in the general technique of verifying properties of logic programs by reasoning about proof terms.


**105/106 Candidate/Span:**	`Span("b'Abstract\xe2\x80\x94Session types are widely accepted as a useful expressive discipline for structuring communications in concurrent and distributed systems.'", sentence=3217, chars=[0,145], words=[0,20])`

**Its parent Sentence's text:**	Abstract—Session types are widely accepted as a useful expressive discipline for structuring communications in concurrent and distributed systems.

**(Solved) Question 1:** the `CandidateExtractor` extracts only spans with length 1. But Each sentence is only getting matched once with one span (try search "sentence=3993", for example). The sentence is good, but we still would want longer span, e.g. half part or the whole of a sentence. 

`Span("b'previous'", sentence=4705, chars=[20,27], words=[4,4])`

**Answer 1:** overwrite `DictionaryMatch._f()`.

=====================================================================================

**(Solved) Question 2:** Several `Span` corresponds to the same sentence, e,g, three `Span` corresponds to `sentence=490`. They wee all included even after we have set `longest_match_only=True`, as these spans all have the longest length.  

**Answer 2:** Perhaps try either (1) filtering by first appearance of document ID; (2) set max_length as the length of the longest sentence length.


### Stopped here as of Wed 06/19 9:39 pm. More to come ... ###

## TODO List

1. Re-format 2K gold label papers. Load them as gold standard. Question: are there any recommended approaches to evaluate segmentation quality, e.g. word overlap? 
2. Our current segmentation strictly segments by sentence boundary, is that errorenous? 



Thanks for reading! 

Some debugging note to memorize (could ignore). 

```
python -m spacy download en
```

Current issue: parser does not parse *by periods*. Sentence count is significantly fewer than expected! 
Potential fix: https://github.com/explosion/spaCy/issues/93

======= Some more debugging log here (not necessary, could skip reading) ======
~~~~
Xins-MacBook-Pro:~ xin$ source activate snorkel
(snorkel) Xins-MacBook-Pro:~ xin$ python
Python 3.6.4 |Anaconda custom (64-bit)| (default, Jan 16 2018, 12:04:33) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)] on darwin
Type "help", "copyright", "credits" or "license" for more information.
>>> import spacey
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
ModuleNotFoundError: No module named 'spacey'
>>> import spacy
>>> spacy.load('en')
<spacy.en.English object at 0x1080e1da0>
>>> model=spacy.load('en')
>>> docs=model.tokenizer('Hello, world. Here are two sentences.')
>>> for sent in docs.sents:
...     pritn(sent.text)
... 
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "spacy/tokens/doc.pyx", line 439, in __get__ (spacy/tokens/doc.cpp:9808)
ValueError: Sentence boundary detection requires the dependency parse, which requires data to be installed. For more info, see the documentation: 
https://spacy.io/docs/usage

>>> for sent in docs.sents:
...     print(sent.text)
... 
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "spacy/tokens/doc.pyx", line 439, in __get__ (spacy/tokens/doc.cpp:9808)
ValueError: Sentence boundary detection requires the dependency parse, which requires data to be installed. For more info, see the documentation: 
https://spacy.io/docs/usage

>>> from spacy.en import English
>>> nlp = English()
>>> doc = nlp(raw_text)
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name 'raw_text' is not defined
>>> raw_text='Hello, world. Here are two sentences.'
>>> doc = nlp(raw_text)
>>> sentences = [sent.string.strip() for sent in doc.sents]
>>> sentences
['Hello, world.', 'Here are two sentences.']
>>> model(raw_text)
Hello, world. Here are two sentences.
>>> docs=model(raw_text)
>>> docs.sents
<generator object at 0x14ad31948>
>>> docs=model(raw_text)
>>> sentences = [sent.string.strip() for sent in docs.sents]
>>> sentences
['Hello, world.', 'Here are two sentences.']
>>> 
~~~~
